# First steps in PySpark 

In this notebook we will learn the fundamentals of functional programming, as well as the basic abstraction of a distributed object in Spark, the RDD. The notebook has been divided into two parts:

Part 1: map/reduce basics

Part 2: Work with RDD and Pair RDD abstractions 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

# Part 1: map/reduce basics

![Hadoop Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Hadoop_logo.svg/220px-Hadoop_logo.svg.png)
# **Apache Hadoop (MapReduce)**

It is an open source software framework written in Java for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures (of individual machines, or racks of machines) are common and thus should be automatically handled in software by the framework.

The core of Apache Hadoop consists of a storage part (Hadoop Distributed File System (HDFS)) and a processing part (MapReduce). Hadoop splits files into large blocks and distributes them amongst the nodes in the cluster. To process the data, Hadoop MapReduce transfers packaged code for nodes to process in parallel, based on the data each node needs to process. This approach takes advantage of data locality — nodes manipulating the data that they have on hand — to allow the data to be processed faster and more efficiently than it would be in a more conventional supercomputer architecture that relies on a parallel file system where computation and data are connected via high-speed networking.

![Map/shuffle/reduce](https://image.slidesharecdn.com/datasciencewebinar061312-120618165932-phpapp02/95/how-salesforcecom-uses-hadoop-41-728.jpg?cb=1340920667)

Since data and computation are distributed, we should avoid the use of variables, i.e. mutable data. Thus, in contrast to impertaive programming, we shall use the functional approach (lambda calculus).

### The goal of the following excercises is to understand basic lambda calculus with python.

### (1a) Functional programming in Python

So, what is Functional Programming? From Wikipedia: 

« …a  programing paradigm that treats computation as the evaluation of mathematical functions and **avoids changing-state and mutable  data**.»

It´s based upon Lambda calculus, wich consist of:
 * Function definition (declaration of expressions)
 * Function application (evaluation of those expressions)
 * Recursion (iteration)

We have already used this in python!!! :)

Recall the typical "lambda x: x+1" we have been using as the first argument of map, reduce and filter methods:
 * **map** maps each value in the input collection to a different value. It´s just the classical mathematical funciton we are used to!
 * **reduce** takes two values from the input collection and returns a new value (of the same type) by appliying a commutative operation to them. 
 * **filter** filters the elements in the input collection according to a certain (boolean) criteria.
 

**Mapping**

`map` is a Higher Order Function (HOF) that takes a function f and a sequence and returns a new sequence formed by applying f to each element in the original sequence. 

![map](https://cosminpupaza.files.wordpress.com/2015/10/map.png?w=505)

In [1]:
import math

math.log1p(2)

1.0986122886681096

In [2]:
list(map(math.log1p, range(5)))

[0.0,
 0.6931471805599453,
 1.0986122886681096,
 1.3862943611198906,
 1.6094379124341003]

In [3]:
math.log1p

<function math.log1p>

Very often we write lambdas, anonymous functions, to use in defining simple transformations with `map`:


In [4]:
list(map(lambda x: x + 2, [4, 8 , 9]))

[6, 10, 11]

In [5]:
vehicles = ['coche', 'moto', 'bici']

list(map(lambda word: word + 's', vehicles))

['coches', 'motos', 'bicis']

In [6]:
def plural(word):
    return word + 's'

We can also use named functions, of course. Note that we pass them as arguments to `map`, not execute them. You can think of them as tools we give to `map` for it to use.

In [7]:
list(map(plural, vehicles))

['coches', 'motos', 'bicis']

Consider the following very common pattern: create empty list, append transformed elements one by one:

In [8]:
result = []

for word in vehicles:
    result.append(plural(word))
    
result

['coches', 'motos', 'bicis']

We can abstract over every possible list by defining a function that takes the list as an argument:

In [9]:
def pluralize_list(input_list):

    result = []

    for word in input_list:
        result.append(plural(word))

    return result
    
pluralize_list(['mama', 'papa'])

['mamas', 'papas']

Consider this other piece of code: we are doing a very similar thing, but with a different function. A lot of the code is common between the two little programs.

Wouldn't it be convenient if we could abstract over every function, as we did over every list?

We can, taking the function to be applied as an argument. This is because Python treats functions as first-class objects, so they can be used as arguments to other functions.

In [10]:
def process_list(f, input_list):
    result = []

    for word in input_list:
        result.append(f(word))

    return result
    
process_list(len, vehicles)

[5, 4, 4]

That function we just wrote, `process_list`, is exactly what `map` is

**Filtering**

`filter` encodes another very common pattern: building a sequence from some of the elements of an input sequence, deciding whether to include each based on the result of evaluating a function, often called the `predicate`, on each of the elements.


![filter](https://cosminpupaza.files.wordpress.com/2015/11/filter.png?w=405)

In [11]:
list(filter(lambda x: x % 2 == 0, range(10)))

[0, 2, 4, 6, 8]

Again, we can use either lambdas or named functions.

In [12]:
def iseven(number):
    return number % 2 == 0

list(filter(iseven, range(10)))

[0, 2, 4, 6, 8]

#### Exercise

write a filter to extract those words that have more than 3 letters from `collection_of_strings`

In [13]:
collection_of_strings = ['tea', 'coffee', 'chai', '7up', 'coca-cola']

list(filter(lambda word: len(word) > 3, collection_of_strings))

['coffee', 'chai', 'coca-cola']

#### Exercise

Write a filter to extract prime numbers from the following list of integers:

```python
[12, 17, 19, 18, 23, 24]
```

You will need to write a function that determines whether a single number is a prime or not.

In [14]:
def isprime(n):
    for candidate in range(2, n):
        if n % candidate == 0:
            return False
    
    return True

isprime(10)

False

In [15]:
numbers = [12, 17, 19, 18, 23, 24] 

list(filter(isprime, numbers))

[17, 19, 23]

`map` and `filter` are not  used explicitly very often in Python, in part because of their cumbersome syntax. 

Nevertheless, there is a well-known feature in Python that is in fact just a convenient way to write `map`s and `filter`s: **list comprehensions**:

In [16]:
[ x ** 2 for x in range(20) if isprime(x) ] 

[0, 1, 4, 9, 25, 49, 121, 169, 289, 361]

The above list comprehension is exactly equivalent to:

In [17]:
list(map(lambda x: x ** 2, (filter(isprime, range(20)))))

[0, 1, 4, 9, 25, 49, 121, 169, 289, 361]

Which is a pipeline of two steps: first a `filter`, then a `map`

**Reducing** 

`reduce` is the third basic foundation of functional programming. Reduce uses a function, often called the `combiner`, to transform a sequence of elements of type T (that is, any type) into a single T. The combiner must take 2 Ts and return only one T.


Recall it must be commutative! Think about the importance of this when parallelizing computations

![reduce](https://cosminpupaza.files.wordpress.com/2015/11/reduce.png?w=500)

In [18]:
from functools import reduce

reduce(lambda x, y: x + y, numbers)

113

#### Exercise

Write a function_x such that, when applied with `reduce`, that reduce returns the highest number in a list. Don't use the built-in `max`!

In [19]:
def function_x(a: int, b: int):
    if a > b:
        return a
    else:
        return b

In [20]:
numbers

[12, 17, 19, 18, 23, 24]

In [21]:
reduce(function_x, numbers)

24

## (1b) Exercise: Calculate the mean of a collection of real numbers using map/reduce
Recall:

$$\bar x = \frac{\sum_{i=1}^{N} x_i}{N} $$

It´s straightforward to do this with python built-in mehots sum() and len(). However, how would you do that with map/reduce? We have already shown how to sum the elements of an array. Thus, you have to calculate the length of the array. For this:
 * Create another array of the same size, consisting of 1s.
 * Sum the elements of that array

#### First part

* Do a reduce to do the sum, and a different map-reduce to get the length

In [22]:
numbers

[12, 17, 19, 18, 23, 24]

In [23]:
total = reduce(lambda x, y: x + y, numbers)
total

113

In [24]:
length = reduce(lambda x, y: x + y, map(lambda n: 1, numbers))
length

6

That was also a pipeline of two steps. It's equivalent to:

In [25]:
ones = map(lambda n: 1, numbers)
reduce(lambda x, y: x + y, ones)

6

#### Combine them in one pass

Think about what it would mean to finish one computation and only then do another in a distributed environment.

Combine both in one pass:

In [26]:
(1,2) + (4, 5) # This is not what we want

(1, 2, 4, 5)

In [27]:
def tuple_combiner(left, right):
    return (left[0] + right[0], left[1] + right[1])

In [28]:
reduce(tuple_combiner, map(lambda n: (n, 1), numbers))

(113, 6)

In [29]:
total, count = reduce(tuple_combiner, map(lambda n: (n, 1), numbers))

mean = total / count

## (1c) Exercise: Calculate the standard deviation of a collection of real numbers

We'll use the typical definition of the standard deviation:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1}$$

For this, use the *mean* and *count* variables from the previous excercise.

In standard idiomatic Python we'd do it like this:

In [30]:
...

Ellipsis

With `map` and `reduce`:

In [31]:
from operator import add

sq_diffs = list(map(lambda x: (x - mean) ** 2, numbers))
sum_sq_diffs = reduce(lambda x, y: x + y, sq_diffs) 
sum_sq_diffs = reduce(add, sq_diffs) # Same thing, but with the add(a, b) function
sum_sq_diffs

94.83333333333334

#### Calculate the standard deviation in one pass.

Again, think about what it would mean to finish one computation and only then do another in a distributed environment.


If we can, we should avoid broadcasting variables all over the cluster. In this case, the following alternative definition of standard deviation can come in handy:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1} =
\frac{\sum_{i=1}^{N} (x_i^2+{\bar x}^2-2x_i\bar x)}{N-1} =
\frac{1}{N-1}\left(\sum_i x_i^2-N\bar x^2\right)$$



In [32]:
def tuple3_combiner(left, right):
    return (left[0] + right[0], left[1] + right[1], left[2] + right[2])

sum_sq, total, count = reduce(tuple3_combiner, map(lambda x: (x**2, x, 1), numbers))
mean = total / count
var = ( sum_sq - count * mean ** 2) / (count - 1)
mean, count, var

(18.833333333333332, 6, 18.966666666666697)

In [33]:
cube = lambda x: pow(x, 3)

In [34]:
cube(5)

125

#### Side note

In Python 2, map and filter returned lists. In Python 3, they return generators, which are lazy collections. They are somewhat similar to files in that they can be depleted of elements after iterating through them.

## (1d) Twe 'word-count' problem: creating histograms
Given a set of keys in an input collection, calculate the frequency of each key. 

In order to understand better how map/reduce works, we will implement this simple calculation in several forms.

We are going to use as test case a list of elements chosen from a small set:

In [35]:
import random
import pandas as pd

random.seed(7)
stations = random.choices(['Esperanza', 'Arguelles', 'Atocha', 'Prosperidad', 'Chamartin'], k=100)
pd.Series(stations).value_counts()

Esperanza      24
Atocha         23
Arguelles      19
Chamartin      18
Prosperidad    16
dtype: int64

### (1d.1) Simple approach

 * Start with an empty dict
 * If a new key is not present in the dict, create it.
 * Otherwise, increase the frequency of the key by one.

In [36]:
result = {}

for station in stations:
    if station not in result:
        result[station] = 1
    else:
        result[station] += 1
        
result

{'Arguelles': 19,
 'Atocha': 23,
 'Chamartin': 18,
 'Esperanza': 24,
 'Prosperidad': 16}

### (1d.2) Map/reduce

* Recall that *reduce* applies an operation to 2 elements of the same type, and returns another element of that type. Thus, the first thing to do is to map our collection to the type of the output. 
 
* The final result will be a dictionary of words in the vocabulary and counts. Therefore, we need to map each input word to this type.
 
* Then, we have to define a function, the combiner, that combines these dictionaries two at a time.

In [37]:
first_step = list(map(lambda station: {station : 1}, stations))
first_step

[{'Arguelles': 1},
 {'Esperanza': 1},
 {'Prosperidad': 1},
 {'Esperanza': 1},
 {'Atocha': 1},
 {'Arguelles': 1},
 {'Esperanza': 1},
 {'Atocha': 1},
 {'Esperanza': 1},
 {'Atocha': 1},
 {'Esperanza': 1},
 {'Esperanza': 1},
 {'Atocha': 1},
 {'Chamartin': 1},
 {'Esperanza': 1},
 {'Arguelles': 1},
 {'Prosperidad': 1},
 {'Chamartin': 1},
 {'Atocha': 1},
 {'Arguelles': 1},
 {'Chamartin': 1},
 {'Esperanza': 1},
 {'Chamartin': 1},
 {'Arguelles': 1},
 {'Esperanza': 1},
 {'Esperanza': 1},
 {'Arguelles': 1},
 {'Chamartin': 1},
 {'Esperanza': 1},
 {'Atocha': 1},
 {'Prosperidad': 1},
 {'Arguelles': 1},
 {'Atocha': 1},
 {'Esperanza': 1},
 {'Esperanza': 1},
 {'Arguelles': 1},
 {'Prosperidad': 1},
 {'Atocha': 1},
 {'Arguelles': 1},
 {'Atocha': 1},
 {'Atocha': 1},
 {'Arguelles': 1},
 {'Prosperidad': 1},
 {'Prosperidad': 1},
 {'Arguelles': 1},
 {'Atocha': 1},
 {'Atocha': 1},
 {'Chamartin': 1},
 {'Prosperidad': 1},
 {'Arguelles': 1},
 {'Chamartin': 1},
 {'Esperanza': 1},
 {'Atocha': 1},
 {'Prosperidad': 1

#### Exercise

Write a combiner function that combines two dictionaries of the form `Dict[str, int]` with any overlap of keys.

In [38]:
a = {'a': 1}
c = a.copy()

c['a'] = 2
c

{'a': 2}

In [39]:
a

{'a': 1}

In [40]:
from typing import Dict

def dict_combiner(left: Dict[str, int], right: Dict[str, int]) -> Dict[str, int]:
    result = left.copy()
    
    for key in right:
        if key in result:
            result[key] += right[key]
        else:
            result[key] = right[key]
    
    return result

reduce(dict_combiner, first_step)

{'Arguelles': 19,
 'Atocha': 23,
 'Chamartin': 18,
 'Esperanza': 24,
 'Prosperidad': 16}

# Part 2: Spark. Work with RDD and Pair RDD abstractions 

![drawing](https://prateekvjoshi.files.wordpress.com/2015/10/1-main4.png)

# ** Apache Spark**

Apache Spark is an open source cluster computing framework originally developed in the AMPLab at University of California, Berkeley but was later donated to the Apache Software Foundation where it remains today. In contrast to Hadoop's two-stage disk-based MapReduce paradigm, Spark's multi-stage in-memory primitives provides performance up to 100 times faster for certain applications.

![](http://image.slidesharecdn.com/sparkandshark-120620130508-phpapp01/95/spark-and-shark-8-728.jpg?cb=1340197567)

By allowing user programs to load data into a cluster's memory and query it repeatedly, Spark is well-suited to machine learning algorithms.
![](http://spark.apache.org/images/logistic-regression.png)

Spark comes with a number of components that provide flexibility and generality.

<img src="http://spark.apache.org/images/spark-stack.png" alt="Drawing" style="width: 500px;"/>

## Word count


### Hadoop

```java
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}
```


### Spark

```python
text_file = sc.textFile("hdfs://...")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("hdfs://...")
```


## In this part, we keep on working on the word-count example, this time with spark. The basic abstraction of Spark is the Resilient Distributed Dataset (RDD):

#### «RDDs are fault-tolerant, parallel data structures that let users explicitly persist intermediate results in memory, control their partitioning to optimize data placement, and manipulate them using a rich set of operators.»

 * Read only, partitioned collection of records (immutable).
 * Stores the transformations used to build a dataset (its lineage), instead of the data itself. This property ensures fault-tolerance.
 * Users can control partitioning and persistence (caching).
 * RDDs are statically typed.
 * … and yes, everything is written in scala ;p. So you could use learning a bit of it!
 
<img src="http://eng.trueaccord.com/wp-content/uploads/2014/10/scala-logo.png" alt="Drawing" style="width: 200px;"/>

#### We will be trying to understand this abstraction with simple examples, using the [Python API](http://spark.apache.org/docs/latest/api/python/index.html)




### If you are running this notebook in Google Colab

Copy the following to a code cell and run it. It will install and set up Spark for you.

```python
!pip install -q findspark pyspark==3.1.1
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.ui.port", "4050").getOrCreate()  ## For ngrok to tunnel to
```

In [41]:
!pip install -q findspark pyspark==3.1.1
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.ui.port", "4050").getOrCreate()  ## For ngrok to tunnel to

     |████████████████████████████████| 212.3MB 73kB/s 
     |████████████████████████████████| 204kB 22.7MB/s 


In [42]:
spark

### ** (2a) Create a base RDD: parallelize, actions and transformations **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

We use sc.parallelize to convert a standard Python collection into an RDD.

In [49]:
2 + 3

5

In [52]:
sc = spark.sparkContext
rdd = sc.parallelize(range(10000000))
rdd

PythonRDD[9] at RDD at PythonRDD.scala:53

In Spark, `map` is a method of the RDD type.

In [60]:
%%time
from math import sqrt

roots = rdd.map(lambda n: n / 0)

CPU times: user 39 µs, sys: 7 µs, total: 46 µs
Wall time: 51.7 µs


**Nothing has actually happened!**

`parallellize` tells spark to distribute the data, but this is not actually done until we perform some action.

Possible actions include counting, collecting, reducing, taking, etc. Take a look at the [Spark programming guide](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions)


In [61]:
%%time

roots.sum()

Py4JJavaError: ignored

#### Exercise

Calculate, from the rdd generated from list_of_cakes, how many cakes we have whose name does **not** start with an 's'

A very common way to write these chained operations in Spark is to put each on a line. Due to Python's syntax, we need to escape the newline character with a backslash.

### Actions and Transformations

There are two main types of operations in Spark:
[Actions](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions) and [Transformations](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#transformations).

**Transformations** produce an RDD. Some of the most important are `map`, `filter`, and `reduceByKey`. `sc.parallelize` and `sc.textFile` are not technically Transformations but you can think of them as such in one very important respect: they are *lazyly evaluated*. That is, when we perform a Transformation, we are only describing the operation to be performed at some point in the future. They attach a node to the execution graph.

**Actions** are implemented as methods on an RDD, and return an object of a type **that is not an RDD**. When we perform an Action, we give the order to execute the previously described transformations: we trigger the execution of the graph. Some of the most important are `reduce`, `collect`, `take`, `takeOrdered`, and `count`.

### **(2b) Persisting and the RDD lineage**

So far, we have seen that Spark RDDs are *lazyly evaluated*, i.e. nothing is actually done until an action is performed. In the RDD, the set of transformations to be applied are remembered: this is known as its *lineage*. It has the important consequence of making Spark RDDs *fault tolerant* automatically.

![](http://images.slideplayer.com/14/4499833/slides/slide_10.jpg) 

It might be interesting to store some intermediate results, though: perhaps because we want to apply several different transformations starting from that point, or because we are going to apply an iterative computation (as is customary in machine learning algorithms). For this, Spark has [several ways of persisting](http://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)

StorageLevel cannot be changed after it has been set. We need to `unpersist` first.

### * Accessing the Spark web UI in Google Colab

We need a Spark session with a web UI port other than 4040. In this example we are using 4050.


```python
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
```        
        

### **(2c) Partitioning **

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster.

To get the number of partitions of an RDD, just use `getNumPartitions()` on your RDD. You can change the partitions during RDD creation (with `parallelize(collection,numPartitions)` or `fromTextFile(file,numPartitions)`), or afterwards with methods like `repartition(), coalesce()`, etc.

We can see the partitions using [glom()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=glom#pyspark.RDD.glom): it returns an RDD created by aggregating all elements within each partition into a list.

Partitions are one of the most powerfull concepts in Spark: you can decide how to distribute your data so it can fit in memory, and more importantly, you can perform computations on each partition *before* speaking to other partitions. This can have an enormous impact on performance

### **(2c) Pair RDDs: *grouping* strategies in Spark**

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD, as we did in the map/reduce version of the histogram in Python, section (1d.2).

We can create the pair RDD using the map() transformation with a lambda() function to create a new RDD.

### ** (2c.1) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`. Next, sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

### ** (2c.2)  `reduceByKey` approach **
A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. 

The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

![](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png)

#### Summary: word count in Spark:

## (2d) Apply word count to a file

### ** (2d.1) Load a text file **
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print(15 lines.)

Copy this into a notebook cell:

```
!wget -v http://www.gutenberg.org/files/100/100-0.txt -O shakespeare.txt
```

### ** (2d.2) Capitalization and punctuation **
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

http://www.regular-expressions.info/quickstart.html

https://regex101.com/

### ** (2d.3) Words from lines **
Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/3/library/stdtypes.html#str.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

Check out the Spark Programming Guide for an alternative transformation that helps us here.

### (2d.4) Count the words and show the top 15

We know the drill at this point, don't we? We map to a tuple then `reduceByKey`

We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pair tuples, we need a custom sort function that sorts using the value part of the pair rather than the key.

You'll notice that many of the words are common English words (know as stopwords).

Use our map reduce and and `takeOrdered()` to obtain the fifteen most common words and their counts.

# Practical

## ETL with airline coupon data

Load the data first: coupon data

#### Exercise

Take fields 0, 2, 3, 4, and 6 from each line of coupons

#### Intermission: Stack traces in Spark

Stack traces in pyspark are quite convoluted because we have many layers of processing. Look for the python stack trace contained within the Py4JJavaError message.

#### Exercise

Keep only the amount. Get average, max, min and std

#### All in one go!



#### Exercise

Get stats on ticket amount for all tickets with destination MAD

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

#### Part 1

We need to filter coupons with destination Madrid, and after that group on the origin. 

At this point, we only need the origin, to group on, and the value, to sum

We are now ready to reduce all amounts per origin.

#### Part 2

This is very similar, with two differences: we need to group on the airline, and to calculate averages we need to keep track of both the total amount on the coupons and the number of coupons.

Now we reduce tuples, summing each component of the tuple separately.

This reduceByKey generates an RDD of the form (k, (v1, v2)). We can map it like this:

Or we can use the mapValues method to only process the (v1, v2) part, ignoring the key for processing without discarding it.

And we're ready to take our result.

#### Stretch: 

Get the totals from first origin without using the coupon number

We'll use groupBy this time, because we need to look at every coupon in a ticket in order to identify first origin.

A ResultIterable is a lazy collection, so we can take it and iterate over it, or turn it into a list to materialize it. This is useful to get a test case to test the function we will be writing.

We are going to compare the set of origins to the set of destinations. The one that is in the first but not in the second should be the first origin.

Now we have have our values, so we are ready to get the totals and sort. Before we reduceByKey, we need to reshape our tuples so the first origin is the key.

### Further resources

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf

https://stackoverflow.com/questions/31610971/spark-repartition-vs-coalesce#:~:text=coalesce%20uses%20existing%20partitions%20to,in%20roughly%20equal%20sized%20partitions.
